### Importing packages and modules

In [1]:
# module for building the pyomo model
import pyomo.environ as pe
# module for solving the pyomo model
import pyomo.opt as po

### Create the model

In [2]:
model = pe.ConcreteModel()

Order to build the model:
1. Sets
1. Parameters
1. Variables
1. Objective function
1. Constraints

#### Sets




In [3]:
N_ROW = 5  
N_COL = 5
MAX_STEPS = N_ROW * N_COL
N_TRUCKS = 2

In [34]:
model.row = pe.Set(initialize=[i for i in range(N_ROW)])
model.column = pe.Set(initialize=[j for j in range(N_COL)])
model.truck = pe.Set(initialize=[k for k in range(N_TRUCKS)])
model.step = pe.Set(initialize=[i for i in range(MAX_STEPS)])

#### Parameters

In [35]:
ND = 1 # Number of deposits

In [36]:
empty_grid = {}

# Iterate over the rows and columns of the 3x3grid
for i in range(N_ROW):
    for j in range(N_COL):
        empty_grid[(i, j)] = 0

$CP(i,j)$: Cantidad de basura que hay en la parada regular en (i,j)

In [37]:
# Total 10 kilos
dirt_regular_stops = empty_grid
stop_position_1 = (1,2)
dirt_regular_stops[stop_position_1] = 3
stop_position_2 = (4,2)
dirt_regular_stops[stop_position_2] = 5
stop_position_3 = (0,4)
dirt_regular_stops[stop_position_3] = 2

model.dirt_regular_stops = pe.Param(model.row, model.column, initialize = dirt_regular_stops)

$D(i,j)$: Depósito en (i,j)

In [38]:
deposit = empty_grid
deposit_position = (0, 0)
deposit[deposit_position] = 1

model.deposit = pe.Param(model.row, model.column, initialize=deposit)

$C_k$: capacidad del vehículo k

In [39]:
capacity = 10.0

model.capacity = pe.Param(initialize = capacity)

#### Variables



In [40]:
# Truck route, ones and zeros to see where it passes
model.r = pe.Var(model.row, model.column, model.step, within=pe.Binary)

# Amount of garbage in the truck
model.b = pe.Var(model.step, within=pe.NonNegativeReals)

# Binary variable indicating if all the garbage has been collected
model.all_dirt_collected = pe.Var(model.step, within=pe.Binary)

# Number of steps taken to return to the depot
model.return_step = pe.Var(within=pe.NonNegativeIntegers, bounds=(0, MAX_STEPS))

# Mark if the garbage from a cell has been collected at each step
model.is_collected = pe.Var(model.row, model.column, model.step, domain=pe.Binary)


#### Objective Function


In [41]:
# Objective function: Minimize the return steps, considering returning to the depot as soon as possible,
# collecting all the garbage, since the depot can only be visited twice and the second visit can only happen
# once all the garbage has been collected

model.obj = pe.Objective(
    expr=model.return_step, 
    sense=pe.minimize
)

#### Constraints

In [42]:
# Remove previous constraints
model.del_component('constraints')
model.constraints = pe.ConstraintList()

# 1. The truck starts at the depot
model.constraints.add(model.r[deposit_position[0], deposit_position[1], 0] == 1)

# 2. Manhattan movement constraint (moves only to adjacent cells)
for s in model.step:
    for i in model.row:
        for j in model.column:
            if s > 0:
                model.constraints.add(
                    model.r[i, j, s] <= 
                    (model.r[i-1, j, s-1] if i > 0 else 0) +  # Up
                    (model.r[i+1, j, s-1] if i < N_ROW - 1 else 0) +  # Down
                    (model.r[i, j-1, s-1] if j > 0 else 0) +  # Left
                    (model.r[i, j+1, s-1] if j < N_COL - 1 else 0)  # Right
                )

# 3. Constraint of only one cell occupied at each time step
for s in model.step:
    model.constraints.add(
        sum(model.r[i, j, s] for i in model.row for j in model.column) == 1
    )

# 4. Truck capacity constraint
for s in model.step:
    model.constraints.add(
        sum(model.dirt_regular_stops[i, j] * model.r[i, j, s] for i in model.row for j in model.column) <= model.capacity
    )

# 5. Constraint to update the truck load when collecting garbage
# Restrict that the load only accumulates the first time a garbage cell is visited
for s in model.step:
    if s > 0:
        model.constraints.add(
            model.b[s] == model.b[s-1] + sum(
                model.dirt_regular_stops[i, j] * model.r[i, j, s] * (1 - model.is_collected[i, j, s-1])
                for i in model.row for j in model.column
            )
        )
    else:
        model.constraints.add(model.b[s] == 0)  # Initially empty

# 6. Constraint to ensure that a garbage cell is marked as collected only once
for s in model.step:
    for i in model.row:
        for j in model.column:
            if s > 0:
                # is_collected is activated only if the cell is visited
                model.constraints.add(
                    model.is_collected[i, j, s] >= model.r[i, j, s]
                )
                # Once activated, it remains in subsequent steps
                model.constraints.add(
                    model.is_collected[i, j, s] >= model.is_collected[i, j, s-1]
                )

# 7. Activate 'all_dirt_collected' when all garbage has been collected
BIG_M = 10000  
small_m = 1*10**-4

# Can be 0 or 1 if it is 0
# It is 0 if it is != 0
for s in model.step:
    model.constraints.add(
        BIG_M * (1 - model.all_dirt_collected[s]) >= sum(model.dirt_regular_stops[i, j] for i in model.row for j in model.column) - model.b[s]
    )

# It is 1 if it is 0
# Can be 0 or 1 if it is != 0
for s in model.step:
    model.constraints.add(
        small_m * (1-model.all_dirt_collected[s]) <= sum(model.dirt_regular_stops[i, j] for i in model.row for j in model.column) - model.b[s]
    )

# 8. The depot can only be visited twice
model.constraints.add(
    sum(model.r[deposit_position[0], deposit_position[1], s] for s in model.step) == 2
)

# 9. Constraint to ensure that the second visit to the depot is only after all garbage has been collected
for s in model.step:
    if s > 0:
        model.constraints.add(
            model.r[deposit_position[0], deposit_position[1], s] <= model.all_dirt_collected[s]
        )
        
# 10. Minimization of the number of steps back to the depot
model.constraints.add(
    model.return_step == sum(s * model.r[deposit_position[0], deposit_position[1], s] for s in model.step)
)

In [43]:
solver = po.SolverFactory('gurobi')
results = solver.solve (model, tee=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-03
Read LP format model from file C:\Users\mtfra\AppData\Local\Temp\tmp13nyh6vn.pyomo.lp
Reading time = 0.01 seconds
x1: 1928 rows, 1301 columns, 5820 nonzeros
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1260P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1928 rows, 1301 columns and 5820 nonzeros
Model fingerprint: 0xdb7bb3cc
Model has 24 quadratic constraints
Variable types: 25 continuous, 1276 integer (1275 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+04]
  QMatrix range    [2e+00, 5e+00]
  QLMatrix range   [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 3e+01]
  RHS range        [1e+00, 1e+04]
Presolve removed 1495 rows and 966 columns
Presolve time: 0.01s
Presolved: 503 rows, 355 colum

In [44]:
# Extract the route from model.r
route = []

# Iterate over each truck, each cell, and each step (if applicable)
for s in range(int(model.return_step.value) + 1):
    for i in model.row:
        for j in model.column:
            if pe.value(model.r[i, j, s]) > 0.5:  # Check if the variable is 1
                route.append((i, j, s))

# Display the route
print("Truck route:")
counter = 0
for point in route:
    print(f"Step {point[2]}: Truck in cell {point[0], point[1]} with a garbage amount: {model.b[point[2]].value}")

Truck route:
Step 0: Truck in cell (0, 0) with a garbage amount: 0.0
Step 1: Truck in cell (0, 1) with a garbage amount: 0.0
Step 2: Truck in cell (0, 2) with a garbage amount: 0.0
Step 3: Truck in cell (0, 3) with a garbage amount: 0.0
Step 4: Truck in cell (0, 4) with a garbage amount: 2.0
Step 5: Truck in cell (0, 3) with a garbage amount: 2.0
Step 6: Truck in cell (1, 3) with a garbage amount: 2.0
Step 7: Truck in cell (1, 2) with a garbage amount: 5.0
Step 8: Truck in cell (2, 2) with a garbage amount: 5.0
Step 9: Truck in cell (3, 2) with a garbage amount: 5.0
Step 10: Truck in cell (4, 2) with a garbage amount: 10.0
Step 11: Truck in cell (3, 2) with a garbage amount: 10.0
Step 12: Truck in cell (2, 2) with a garbage amount: 10.0
Step 13: Truck in cell (1, 2) with a garbage amount: 10.0
Step 14: Truck in cell (0, 2) with a garbage amount: 10.0
Step 15: Truck in cell (0, 1) with a garbage amount: 10.0
Step 16: Truck in cell (0, 0) with a garbage amount: 10.0


In [45]:
import time  

# Colores ANSI
RESET = "\033[0m"
RED = "\033[31m"  # Rojo para el paso actual
CYAN = "\033[36m"  # Cian para celdas previamente visitadas
GREEN = "\033[32m"  # Verde para celdas con basura

# Matriz con celdas vacías y cantidades de basura
matriz = [[f"{GREEN}{model.dirt_regular_stops[i, j]}{RESET}" if model.dirt_regular_stops[i, j] > 0 else " " 
           for j in range(N_COL)] 
          for i in range(N_ROW)]

print("Recorrido del camión paso a paso:\n")

# Lista para guardar las celdas visitadas
visitadas = [[False for _ in range(N_COL)] for _ in range(N_ROW)]

# Recorrido del camión paso por paso
for s in range(int(model.return_step.value) + 1):
    # Crear una copia para marcar el paso actual sin modificar la matriz original
    matriz_temp = [fila.copy() for fila in matriz]

    # Buscar la celda visitada en este paso
    for i in model.row:
        for j in model.column:
            if model.r[i, j, s].value == 1:
                matriz_temp[i][j] = f"{RED}X{RESET}"  # Marcar la celda actual en rojo
                visitadas[i][j] = True  # Marcar la celda como visitada

    # Actualizar celdas previamente visitadas a cian
    for i in range(N_ROW):
        for j in range(N_COL):
            if visitadas[i][j] and matriz_temp[i][j] != f"{RED}X{RESET}":  
                matriz_temp[i][j] = f"{CYAN}X{RESET}"

    # Mostrar la matriz actualizada
    for fila in matriz_temp:
        print("+---" * N_COL + "+")
        print("| " + " | ".join(fila) + " |")
    print("+---" * N_COL + "+\n")
    
    # Esperar antes de continuar al siguiente paso
    time.sleep(0.5) 

Recorrido del camión paso a paso:

+---+---+---+---+---+
| X |   |   |   | 2 |
+---+---+---+---+---+
|   |   | 3 |   |   |
+---+---+---+---+---+
|   |   |   |   |   |
+---+---+---+---+---+
|   |   |   |   |   |
+---+---+---+---+---+
|   |   | 5 |   |   |
+---+---+---+---+---+

+---+---+---+---+---+
| X | X |   |   | 2 |
+---+---+---+---+---+
|   |   | 3 |   |   |
+---+---+---+---+---+
|   |   |   |   |   |
+---+---+---+---+---+
|   |   |   |   |   |
+---+---+---+---+---+
|   |   | 5 |   |   |
+---+---+---+---+---+

+---+---+---+---+---+
| X | X | X |   | 2 |
+---+---+---+---+---+
|   |   | 3 |   |   |
+---+---+---+---+---+
|   |   |   |   |   |
+---+---+---+---+---+
|   |   |   |   |   |
+---+---+---+---+---+
|   |   | 5 |   |   |
+---+---+---+---+---+

+---+---+---+---+---+
| X | X | X | X | 2 |
+---+---+---+---+---+
|   |   | 3 |   |   |
+---+---+---+---+---+
|   |   |   |   |   |
+---+---+---+---+---+
|   |   |   |   |   |
+---+---+---+---+---+
|   |   | 5 |   |   |
+---+---+---+---